In [ ]:
###Скрипт проверяет, есть ли дублирование номеров глав и текста, глав в целом, есть ли пропущенные главы. ###
###Если в тексте есть пара абзацев повторяющихся, он их не найдет. ###

In [ ]:
from docx import Document
import re
from collections import defaultdict

# Словарь для преобразования чисел из текстового формата
TEXT_NUMBERS = {
    'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6,
    'seven': 7, 'eight': 8, 'nine': 9, 'ten': 10, 'eleven': 11, 'twelve': 12,
    'thirteen': 13, 'fourteen': 14, 'fifteen': 15, 'sixteen': 16,
    'seventeen': 17, 'eighteen': 18, 'nineteen': 19, 'twenty': 20,
    'thirty': 30, 'forty': 40, 'fifty': 50, 'sixty': 60, 'seventy': 70,
    'eighty': 80, 'ninety': 90, 'hundred': 100
}

def word_to_number(word):
    """Преобразует текстовое число в числовое значение."""
    words = re.split(r'[\s\-]+', word.lower())  # Разбиваем по пробелам и дефисам
    number = 0
    temp = 0

    for w in words:
        if w in TEXT_NUMBERS:
            scale = TEXT_NUMBERS[w]
            if scale == 100:  # Обработка сотен, например "one hundred"
                temp *= scale
            else:
                temp += scale
        else:
            return None  # Если слово не распознано, возвращаем None

    number += temp
    return number

def extract_chapter_number(text):
    """Извлекает номер главы из текста."""
    # Проверяем числовой формат, например, "Chapter 1"
    numeric_match = re.match(r'Chapter\s+(\d+)', text, re.IGNORECASE)
    if numeric_match:
        return int(numeric_match.group(1))
    
    # Проверяем текстовый формат, например, "Chapter One" или "Chapter Thirty-one"
    text_match = re.match(r'Chapter\s+([\w\s\-]+)', text, re.IGNORECASE)
    if text_match:
        word = text_match.group(1).strip()
        return word_to_number(word)
    return None

def extract_chapters(docx_file):
    """Извлекает главы и их содержимое из .docx файла."""
    doc = Document(docx_file)
    chapters = []
    current_chapter = None
    current_text = []

    for para in doc.paragraphs:
        chapter_num = extract_chapter_number(para.text.strip())
        if chapter_num is not None:
            if current_chapter is not None and current_text:
                chapters.append((current_chapter, "\n".join(current_text).strip()))
            current_chapter = chapter_num
            current_text = []
        else:
            current_text.append(para.text.strip())

    if current_chapter is not None and current_text:
        chapters.append((current_chapter, "\n".join(current_text).strip()))

    return chapters

def check_duplicate_chapters(chapters):
    chapter_numbers = [chap[0] for chap in chapters]
    return {chap for chap in chapter_numbers if chapter_numbers.count(chap) > 1}

def check_text_duplicates(chapters):
    """Проверяет дублирующийся текст в главах с учётом нормализации."""
    text_to_chapter = defaultdict(list)

    for chapter, text in chapters:
        # Нормализуем текст: убираем лишние пробелы и приводим к нижнему регистру
        normalized_text = " ".join(text.split()).lower()
        
        # Добавляем номер главы в список глав с таким же текстом
        text_to_chapter[normalized_text].append(chapter)

    # Фильтруем только те записи, где один и тот же текст встречается более одного раза
    return {text: chaps for text, chaps in text_to_chapter.items() if len(chaps) > 1}

def check_chapter_order(chapters):
    chapter_numbers = [chap[0] for chap in chapters]
    return [(chapter_numbers[i], chapter_numbers[i+1]) 
            for i in range(len(chapter_numbers) - 1) if chapter_numbers[i] > chapter_numbers[i + 1]]

def check_missing_chapters(chapters):
    """Проверяет пропущенные главы в последовательности."""
    chapter_numbers = sorted([chap[0] for chap in chapters])
    missing_chapters = [num for num in range(chapter_numbers[0], chapter_numbers[-1] + 1) 
                        if num not in chapter_numbers]
    return missing_chapters

def analyze_book(docx_file):
    chapters = extract_chapters(docx_file)

    print("Список найденных глав:")
    for chapter, text in chapters:
        print(f"Chapter {chapter}: {text[:50]}...")
    
    duplicate_chapters = check_duplicate_chapters(chapters)
    duplicate_texts = check_text_duplicates(chapters)
    unordered_chapters = check_chapter_order(chapters)
    missing_chapters = check_missing_chapters(chapters)

    print("\nРезультаты проверок:")
    print(f"Найдены повторяющиеся главы: {duplicate_chapters}" if duplicate_chapters else "Повторяющиеся главы не найдены.")
    print(f"Найдены главы с дублирующимся текстом: {duplicate_texts}" if duplicate_texts else "Дублирующийся текст не найден.")
    print(f"Найдены главы с нарушением порядка: {unordered_chapters}" if unordered_chapters else "Нарушений в порядке глав не обнаружено.")
    print(f"Пропущенные главы: {missing_chapters}" if missing_chapters else "Пропущенных глав не найдено.")

# Использование
docx_file = 'werewolf_heartbeat/fulltext.docx'
analyze_book(docx_file)
